In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
import sqlite3
import pandas as pd

[a]

In [3]:
conn_hedge = sqlite3.connect("hedgefund.db")
hedgefund_df = pd.read_sql_query("SELECT * FROM hf_wide", conn_hedge)
hedgefund_df.head()

,Year,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,YTD
0,1997,0.0362,0.0118,-0.0099,0.0049,0.0406,0.0311,0.0470,0.0052,0.0405,-0.0127,-0.0019,0.0129,0.2235
1,1998,-0.0019,0.0376,0.0354,0.0115,-0.0168,0.0023,-0.0046,-0.0781,0.0105,0.0217,0.0378,0.0295,0.0823
2,1999,0.0223,-0.0084,0.0348,0.0492,0.0115,0.0413,0.0082,0.0032,0.0074,0.0190,0.0533,0.0773,0.3656
3,2000,0.0095,0.0676,0.0174,-0.0147,-0.0100,0.0326,0.0006,0.0374,-0.0071,-0.0107,-0.0252,0.0220,0.1219
4,2001,0.0304,-0.0144,-0.0111,0.0202,0.0147,0.0051,-0.0057,-0.0018,-0.0258,0.0184,0.0202,0.0171,0.0677


In [8]:
# Reshape data from wide to long format
long_hedgefund_df = pd.melt(hedgefund_df, id_vars='Year', value_vars=hedgefund_df.columns[1:], var_name='Month', value_name='Ret')
long_hedgefund_df = long_hedgefund_df[long_hedgefund_df['Month'] != 'YTD']

# Combine 'Year' and 'Month' into a single column
long_hedgefund_df['YearMonth'] = long_hedgefund_df['Year'].astype(int).astype(str) + long_hedgefund_df['Month'].astype(str)
long_hedgefund_df['Date'] = pd.to_datetime(long_hedgefund_df['YearMonth'], format='%Y%b')
long_hedgefund_df['Date'] = long_hedgefund_df['Date'].dt.strftime('%Y%m')
long_hedgefund_df = long_hedgefund_df.drop(['YearMonth'], axis=1)
long_hedgefund_df['Month'] = pd.to_datetime(long_hedgefund_df['Month'], format='%b').dt.month

# Reorder columns and reset index
long_hedgefund_df = long_hedgefund_df.reset_index(drop=True)
long_hedgefund_df = long_hedgefund_df[['Year','Month','Ret','Date']]

# Sort data by Year
long_hedgefund_df.sort_values(by='Date', inplace=True)

In [9]:
# Connect to the databases and create the tables
conn_out = sqlite3.connect("output.db")
long_hedgefund_df.to_sql("hf_long", conn_out, if_exists="replace", index=False)

# Query the data to make sure it was stored correctly
check_table_hf_long = pd.read_sql_query("SELECT * FROM hf_long", conn_out)
check_table_hf_long.head()

,Year,Month,Ret,Date
0,1997,1,0.0362,199701
1,1997,2,0.0118,199702
2,1997,3,-0.0099,199703
3,1997,4,0.0049,199704
4,1997,5,0.0406,199705


[b]

In [10]:
conn_rf = sqlite3.connect("rfasset.db")
rfasset_df = pd.read_sql_query("SELECT * FROM rf_long", conn_rf)
rfasset_df.head()

,Date,RF
0,192607,0.0022
1,192608,0.0025
2,192609,0.0023
3,192610,0.0032
4,192611,0.0031


In [11]:
# Match RF
match_RF = []
for i, rowi in long_hedgefund_df.iterrows():
    #print(rowi['Date'])
    for j, rowj in rfasset_df.iterrows():
        #print(rowj['Date'])
        if int(rowi['Date']) == rowj['Date']:
            match_RF.append(rowj['RF'])
            #print(rowj['RF'])
match_RF.append(0)

In [13]:
# Merge RF to long_hedgefund_df
merged_df = long_hedgefund_df
merged_df['RF'] = match_RF

In [14]:
# Create the tables
merged_df.to_sql("merged", conn_out, if_exists="replace", index=False)

# Query the data to make sure it was stored correctly
check_table_merged = pd.read_sql_query("SELECT * FROM merged", conn_out)
check_table_merged.head()

,Year,Month,Ret,Date,RF
0,1997,1,0.0362,199701,0.0045
1,1997,2,0.0118,199702,0.0039
2,1997,3,-0.0099,199703,0.0043
3,1997,4,0.0049,199704,0.0043
4,1997,5,0.0406,199705,0.0049
